In [1]:
import dash_bootstrap_components as dbc
import geopandas as gpd
import json
import matplotlib
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests

from dash import Dash, html, dcc, callback, Output, Input, get_asset_url
from PIL import Image
from shapely import union_all

In [2]:
# data input and resources
seguia_logo_img = Image.open("../references/branding/Logo seguia.png")

drought_data_path = '../data/interim/drought_data_info2024.parquet'
drought_data = pd.read_parquet(drought_data_path)
municipal_geometry_path = '../data/raw/muni.pkl'
municipal_geometry = pd.read_pickle(municipal_geometry_path)

estados = gpd.read_file('../../../../../Downloads/destdv1gw/destdv1gw.shp')
estados = estados.sort_values(['ENTIDAD','AREA']).drop_duplicates('ENTIDAD', keep='last')
estados = estados.rename(columns={'CVE_EDO': 'CVE_ENT'})[['ENTIDAD', 'CVE_ENT', 'CAPITAL', 'geometry']]
state_geom_json = json.loads(estados.reset_index(drop=True).to_json())

FileNotFoundError: [Errno 2] No such file or directory: '../data/interim/drought_data_info2024.parquet'

In [ ]:
# configuration
theme = dbc.themes.BOOTSTRAP

zoom_for_mexico = {
    'country': 3.4,
    'state': {}
}

N=drought_data.DROUGHT_INDEX.nunique()

cmap = matplotlib.colormaps.get_cmap('Reds').resampled(N)
colors = [matplotlib.colors.to_hex(cmap(i)) for i in range(N)]

# Process_data
mg_cols = ['CVE_ENT', 'NOM_ENT', 'geometry', 'CVEGEO', 'NOM_MUN', 'AREA']
muni_state_drought_data = drought_data.merge(
    municipal_geometry[mg_cols], 
    left_on='CVE_CONCATENADA', 
    right_on='CVEGEO', 
    how='right'
)
muni_state_drought_data['year'] = muni_state_drought_data.DATE.apply(lambda x:x.year)

mexico_centroid = union_all(estados.geometry).centroid
mexico_lon, mexico_lat = mexico_centroid.x, mexico_centroid.y

year_list = pd.Series(muni_state_drought_data.year.unique())

max_val_drought_index = muni_state_drought_data.groupby(['DATE', 'CVE_ENT']).DROUGHT_INDEX.mean().max()
min_val_drought_index = muni_state_drought_data.groupby(['DATE', 'CVE_ENT']).DROUGHT_INDEX.mean().min()

In [ ]:
# This is information regarding the average drought_index in a year. 
#muni_state_drought_data[muni_state_drought_data.year==year_value].groupby('CVE_ENT').DROUGHT_INDEX.mean().reset_index()

# This is information regarding the proportion of drought_index in a year.
#muni_state_drought_data[muni_state_drought_data.year==year_value].DROUGHT_INDEX.value_counts(normalize=True)

In [ ]:
year_value = 2023
value = 'AGUASCALIENTES'

temp_cve_ent = estados[estados.ENTIDAD == value].CVE_ENT.iloc[0]
df = muni_state_drought_data[
    (muni_state_drought_data.CVE_ENT == temp_cve_ent) &
    (muni_state_drought_data.year  == year_value) 
].groupby("CVE_CONCATENADA").DROUGHT_INDEX.mean().reset_index()

df

fig = go.Figure(
        go.Choroplethmapbox(
            name='State map',
            geojson=state_geom_json,
            ids = df['CVE_CONCATENADA'],
            z=df['DROUGHT_INDEX'],
            zmax=max_val_drought_index,
            zmin=min_val_drought_index,
            locations=df['CVE_CONCATENADA'],
            featureidkey='properties.CVE_CONCATENADA',
            colorscale='reds',
        )
    )
muni_geom_json = json.loads(
    municipal_geometry[
        municipal_geometry.CVEGEO.isin(df.CVE_CONCATENADA)
    ][
        
       ['CVEGEO','geometry']].to_json())

In [ ]:
df = muni_state_drought_data[
        muni_state_drought_data.year==value
    ].groupby('CVE_ENT').DROUGHT_INDEX.mean().reset_index()
fig = go.Figure(
        go.Choroplethmapbox(
            name='Mexico state map',
            geojson=state_geom_json,
            ids = df['CVE_ENT'],
            z=df['DROUGHT_INDEX'],
            zmax=max_val_drought_index,
            zmin=min_val_drought_index,
            locations=df['CVE_ENT'],
            featureidkey='properties.CVE_ENT',
            colorscale='reds',
        )
    )
fig.update_layout(
            mapbox_style='carto-positron',
            mapbox_zoom=zoom_for_mexico['country'], 
            mapbox_center = {'lat': mexico_lat, 'lon': mexico_lon},
        )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
what_happens_in_mexico = """La mitad de la población en méxico atraviesa una sequía.
Hoy estamos pasando por la peor crisis hídrica en tres años. Y parece que todo va a seguir empeorando: 
Las sequías serán más recurrentes, y también serán exacerbadas por el cambio climático, provocando una crisis tanto alimentaria como económica.
Aunque la temporada de ciclones podría traer algo de alivio temporal, la cruda realidad es que la escasez de agua seguirá asolando al país. 
Esta crisis requiere una acción urgente y decidida para evitar un futuro aún más sombrío."""


In [ ]:
app = Dash(
    __name__, 
    external_stylesheets=[theme]
)


app.layout = html.Div([
    html.H1(children='¿Nos estamos quedando sin agua?', style={'textAlign':'center'}),
    html.P(children=what_happens_in_mexico),
    html.H2(children='¿Qué pasa con México?'),
    html.Div([
        html.Div([
            # html.P(children='Aqui va el primer gráfico, es una imagen de Mexico'),
            dcc.Graph(id='mexico-state-map'),

        ], style={'width': '64%', 'display': 'inline-block', 'padding': '0px 20px 20px 20px'}),
        html.Div([
            # html.P(children='Aqui va el segundo grafico de pie. Es la proporción de sequías.')
            dcc.Graph(id='mexico-municipality-piechart')
        ], style={'width': '34%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.Div(
        #html.P(children='Aqui es la informacion del slider para cambiar de fecha. Lorem ipsum dolor sit amet, consectetur adipiscing elit.')
        [dcc.Slider(
            min=year_list.min(),
            max=year_list.max(), 
            step=1, 
            value=int(year_list.median()),
            marks={y:f'{y}' for y in year_list},
            id='date-filter-for-mexico-state-map-and-mexico-municipal-piechart',
        )]
    ),
    html.P(children='Este es el segundo parrafo que explica lo anterior: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam faucibus orci risus, eu vehicula magna vestibulum eu. Integer luctus non eros a aliquam. Nulla magna nisl, cursus sit amet urna et, suscipit vestibulum libero. Mauris vehicula sit amet purus id pharetra. Suspendisse nec turpis dui. Suspendisse semper nisi quis nulla rhoncus bibendum. Aenean ut nisl at risus tincidunt fringilla placerat sit amet leo. Ut a sem non eros euismod elementum non id lorem. Nunc volutpat quam venenatis diam sodales, et scelerisque neque maximus. Pellentesque laoreet diam in elit pretium ullamcorper. Nunc dignissim odio lacus, eu congue enim fermentum vitae.'),
    html.H2(children='Donde vivo, ¿ya valió?'),
    
    # Falta el dropdown
    html.Div([
        dcc.Dropdown(estados.ENTIDAD.unique(), 'AGUASCALIENTES', id='dropdown-selection-state-map')
    ], style={'width': '59%', 'display': 'inline-block'}),
    
    html.Div([
        html.Div([
            #html.P(children='Aqui va el gráfico, es una imagen de el estado seleccionado'),
            dcc.Graph(id='state-municipality-map')
        ], style={'width': '59%', 'display': 'inline-block'}),
        html.Div([
            html.P(children='Aqui va el escrito del municipio, explica a nivel estado que esta pasando y también a nivel municipio: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam faucibus orci risus, eu vehicula magna vestibulum eu. Integer luctus non eros a aliquam. Nulla magna nisl, cursus sit amet urna et, suscipit vestibulum libero.')
        ], style={'width': '39%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.Div(
        [dcc.Slider(0, 20, 5,value=10, id='date-filter-for-state-municipality-map')]
        #html.P(children='Aqui es la informacion del slider para cambiar de fecha. Lorem ipsum dolor sit amet, consectetur adipiscing elit.'),
        #style={'padding': '100px 20px 20px 20px'}

        #dcc.Slider(id=date-filter-for-second-state-municipality-map)
    ),
    html.Div(
        [html.P(children='Aqui va la grafica lineal donde se busca representar el estado del cual se va hablando, y se selecciona el valor de la fecha sombreada')],
        style={'padding': '50px 20px 20px 20px'}
    ),
    html.H2(children='¿Hay una solución?... Pues hicimos un modelo'),
    html.Div([
        html.Div([
            html.P(children='Este es el parrafo que explica el modelo: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam faucibus orci risus, eu vehicula magna vestibulum eu. Integer luctus non eros a aliquam. Nulla magna nisl, cursus sit amet urna et, suscipit vestibulum libero. Mauris vehicula sit amet purus id pharetra. Suspendisse nec turpis dui. Suspendisse semper nisi quis nulla rhoncus bibendum. Aenean ut nisl at risus tincidunt fringilla placerat sit amet leo. Ut a sem non eros euismod elementum non id lorem. Nunc volutpat quam venenatis diam sodales, et scelerisque neque maximus. Pellentesque laoreet diam in elit pretium ullamcorper. Nunc dignissim odio lacus, eu congue enim fermentum vitae.'),

        ], style={'width': '74%', 'display': 'inline-block'}),
        html.Div([
            #html.P(children='Aqui va el logo del modelo: Seguia.')
            html.Img(src=seguia_logo_img)
        ], style={'width': '24%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Div([
            html.H3(children='Predicción '),
        ], style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
            html.H3(children='Realidad '),
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Div([
            #html.P(children='Aquí va una imagen donde se observa la predicción del modelo '),
            dcc.Graph(id='model-prediction-map')
        ], style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
            #html.P(children='Aquí va una imagen donde se observa la realidad de lo que el modelo debió haber predicho.'),
            dcc.Graph(id='model-real_value-map')
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ]),
    html.P(children='Aquí va una explicación donde se habla de lo que se está viendo Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam faucibus orci risus, eu vehicula magna vestibulum eu. Integer luctus non eros a aliquam. Nulla magna nisl, cursus sit amet urna et, suscipit vestibulum libero. Mauris vehicula sit amet purus id pharetra. Suspendisse nec turpis dui. Suspendisse semper nisi quis nulla rhoncus bibendum. Aenean ut nisl at risus tincidunt fringilla placerat sit amet leo. Ut a sem non eros euismod elementum non id lorem. Nunc volutpat quam venenatis diam sodales, et scelerisque neque maximus. Pellentesque laoreet diam in elit pretium ullamcorper. Nunc dignissim odio lacus, eu congue enim fermentum vitae.')
])

In [ ]:
@callback(
    Output('mexico-state-map', 'figure'),
    Input('date-filter-for-mexico-state-map-and-mexico-municipal-piechart', 'value')
)
def plot_mexico_map(value):
    df = muni_state_drought_data[
        muni_state_drought_data.year==value
    ].groupby('CVE_ENT').DROUGHT_INDEX.mean().reset_index()
    fig = go.Figure(
        go.Choroplethmapbox(
            name='Mexico state map',
            geojson=state_geom_json,
            ids = df['CVE_ENT'],
            z=df['DROUGHT_INDEX'],
            zmax=max_val_drought_index,
            zmin=min_val_drought_index,
            locations=df['CVE_ENT'],
            featureidkey='properties.CVE_ENT',
            colorscale='reds',
        )
    )
    fig.update_layout(
            mapbox_style='carto-positron',
            mapbox_zoom=zoom_for_mexico['country'], 
            mapbox_center = {'lat': mexico_lat, 'lon': mexico_lon},
        )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

@callback(
    Output('mexico-municipality-piechart', 'figure'),
    Input('date-filter-for-mexico-state-map-and-mexico-municipal-piechart', 'value')
)
def plot_municipality_pie_chart(value):
    df = muni_state_drought_data[
        muni_state_drought_data.year==value
    ].DROUGHT_INDEX.value_counts(normalize=True).reset_index()
    fig = go.Figure(
    go.Pie(
        labels=df['DROUGHT_INDEX'],
        values=df['proportion'], 
        marker_colors=colors
    ))
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

In [3]:
if __name__ == '__main__':
    app.run()

NameError: name 'app' is not defined